In [1]:
#v3

import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.linear_model import  LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import *
from sklearn import metrics
import numpy as np
import pandas as pd
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import  LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
#from sklearn.cross_validation import KFold
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
import matplotlib.pyplot as plt
import itertools

import string
import nltk
from nltk import PorterStemmer
import re 

import gensim
from gensim import corpora

ps = PorterStemmer()
wn = nltk.WordNetLemmatizer()
predictedProb = 0


class PartyAffiliation():
    
    
    
    # API to check whether the subject(Headline) is present in the 
    # - democrats most used words if the party affiliation is democrat
    # - republicans most used words if the part affiliation is republican
    def partyAffiliationFromHeadline(self, r):
        v = r['subject_str']
        p = r['party_str']
        if (p =='democrat'):
            s2 = set(self.countDemV.get_feature_names())
        if (p =='republican'):
            s2 = set(self.countRepV.get_feature_names())
        if (p != 'democract' and p !='republican'):
            return 1 #'true'        
        if set(v).intersection(s2):
            return 1 #'true'
        else:
            return 0 #'false'

    #API to convert true, mostly-true and half-true to true
    # false, barely-true and pants-fire to false
    def convertMulticlassToBinaryclass(self, r):
        v = r['label']
        if (v == 'true'):
            return 1 #'true'
        if (v == 'mostly-true'):
            return 1 #'true'
        if (v == 'half-true'):
            return 1 #'true'
        if (v == 'barely-true'):
            return 0 #'false'
        if (v == 'false'):
            return 0 #'false'
        if (v == 'pants-fire'):
            return 0 #'false'
            
            
            
    def plot_confusion_matrix(self, cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):

        plt.imshow(cm, interpolation='nearest', cmap=cmap)
        plt.title(title)
        plt.colorbar()
        tick_marks = np.arange(len(classes))
        plt.xticks(tick_marks, classes, rotation=45)
        plt.yticks(tick_marks, classes)

        if normalize:
            cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
            print("Normalized confusion matrix")
        else:
            print('Confusion matrix, without normalization')

        thresh = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            plt.text(j, i, cm[i, j],
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')   
        
        
        
        
    # Functions to preprocess the data ( punctuation/Stop words  and stemmer)
    def clean_stem (self, sent): 
        stopwords = nltk.corpus.stopwords.words('english')
        temp1 ="".join(x for x in sent if x not in string.punctuation)
        temp2 = re.split('\W+',temp1.lower())
        temp3 = [ps.stem(x) for x in temp2 if x not in stopwords]
        return temp3

    # Function to use wordnet lemmatizer 
    def clean_lemma (self, sent): 
        stopwords = nltk.corpus.stopwords.words('english')
        temp1 ="".join(x for x in sent if x not in string.punctuation)
        temp2 = re.split('\W+',temp1.lower())
        temp3 = [wn.lemmatize(x) for x in temp2 if x not in stopwords]
        return temp3
    
    def clean(self, doc):
        stop = nltk.corpus.stopwords.words('english')
        exclude = set(string.punctuation) 
        lemma = wn
        stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
        punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
        normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
        return normalized
    
    def testFunc(self, prob):
        global predictedProb 
        predictedProb += 0.26
        predictedProb += prob

            
    
    def __init__(self):        

        columnNamesPar = ["id", "label", "statement", "subject", "speaker", "speaker_job_title", "state_info", "party_affiliation", "barely_true_counts", "false_counts", "half_true_counts", "mostly_true_counts", "pants_on_fire_counts", "context"]
        dataTrainPar = pd.read_csv('data/liar-train.tsv', sep='\t', header=None, names = columnNamesPar)
        dataValidatePar = pd.read_csv('data/liar-valid.tsv', sep='\t', header=None, names = columnNamesPar)
        dataTestPar = pd.read_csv('data/liar-test.tsv', sep='\t', header=None, names = columnNamesPar)
        
    
        # Remove unwanted columns in the dataset
        columnsToRemovePar = ['id', 'speaker', 'context','speaker_job_title', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts']
        dataTrainPar = dataTrainPar.drop(columns=columnsToRemovePar)
        dataValidatePar = dataValidatePar.drop(columns=columnsToRemovePar)
        dataTestPar = dataTestPar.drop(columns=columnsToRemovePar)
        
        # convert the labels to true and false only
        dataTrainPar['label'] = dataTrainPar.apply(self.convertMulticlassToBinaryclass, axis=1)
        dataValidatePar['label'] = dataValidatePar.apply(self.convertMulticlassToBinaryclass, axis=1)
        dataTestPar['label'] = dataTestPar.apply(self.convertMulticlassToBinaryclass, axis=1)
        
        # display all the party affiliations and show the count of each party 
#         dataTrainPar.groupby('party_affiliation').count()[['state_info']].rename(
#         columns={'state_info': 'count'}).sort_values(
#         'count', ascending=False).reset_index().plot.bar(
#         x='party_affiliation', y='count', figsize=(16, 10), fontsize=18);
        
        # As we are considering only democrat, republican and none (top 3 party affiliations),
        # ignoring other party affiliations
        rowsToRemove = ['Moderate', 'activist', 'business-leader', 'columnist', 'constitution-party', 'democratic-farmer-labor', 'education-official', 'government-body', 'green', 'independent', 'journalist', 'labor-leader', 'liberal-party-canada', 'libertarian', 'nan', 'newsmaker', 'ocean-state-tea-party-action', 'organization', 'state-official', 'talk-show-host', 'tea-party-member']

        dataTrainPar = dataTrainPar[dataTrainPar.party_affiliation != 'Moderate']
        dataTrainPar = dataTrainPar[dataTrainPar.party_affiliation != 'activist']
        dataTrainPar = dataTrainPar[dataTrainPar.party_affiliation != 'business-leader']
        dataTrainPar = dataTrainPar[dataTrainPar.party_affiliation != 'columnist']
        dataTrainPar = dataTrainPar[dataTrainPar.party_affiliation != 'constitution-party']
        dataTrainPar = dataTrainPar[dataTrainPar.party_affiliation != 'democratic-farmer-labor']
        dataTrainPar = dataTrainPar[dataTrainPar.party_affiliation != 'education-official']
        dataTrainPar = dataTrainPar[dataTrainPar.party_affiliation != 'government-body']
        dataTrainPar = dataTrainPar[dataTrainPar.party_affiliation != 'green']
        dataTrainPar = dataTrainPar[dataTrainPar.party_affiliation != 'independent']
        dataTrainPar = dataTrainPar[dataTrainPar.party_affiliation != 'journalist']
        dataTrainPar = dataTrainPar[dataTrainPar.party_affiliation != 'labor-leader']
        dataTrainPar = dataTrainPar[dataTrainPar.party_affiliation != 'liberal-party-canada']
        dataTrainPar = dataTrainPar[dataTrainPar.party_affiliation != 'libertarian']
        dataTrainPar = dataTrainPar[dataTrainPar.party_affiliation != 'nan']
        dataTrainPar = dataTrainPar[dataTrainPar.party_affiliation != 'newsmaker']
        dataTrainPar = dataTrainPar[dataTrainPar.party_affiliation != 'ocean-state-tea-party-action']
        dataTrainPar = dataTrainPar[dataTrainPar.party_affiliation != 'organization']
        dataTrainPar = dataTrainPar[dataTrainPar.party_affiliation != 'state-official']
        dataTrainPar = dataTrainPar[dataTrainPar.party_affiliation != 'talk-show-host']
        dataTrainPar = dataTrainPar[dataTrainPar.party_affiliation != 'tea-party-member']

        # As we are considering only democrat, republican and none (top 3 party affiliations),
        # ignoring other party affiliations

        dataTestPar = dataTestPar[dataTestPar.party_affiliation != 'Moderate']
        dataTestPar = dataTestPar[dataTestPar.party_affiliation != 'activist']
        dataTestPar = dataTestPar[dataTestPar.party_affiliation != 'business-leader']
        dataTestPar = dataTestPar[dataTestPar.party_affiliation != 'columnist']
        dataTestPar = dataTestPar[dataTestPar.party_affiliation != 'constitution-party']
        dataTestPar = dataTestPar[dataTestPar.party_affiliation != 'democratic-farmer-labor']
        dataTestPar = dataTestPar[dataTestPar.party_affiliation != 'education-official']
        dataTestPar = dataTestPar[dataTestPar.party_affiliation != 'government-body']
        dataTestPar = dataTestPar[dataTestPar.party_affiliation != 'green']
        dataTestPar = dataTestPar[dataTestPar.party_affiliation != 'independent']
        dataTestPar = dataTestPar[dataTestPar.party_affiliation != 'journalist']
        dataTestPar = dataTestPar[dataTestPar.party_affiliation != 'labor-leader']
        dataTestPar = dataTestPar[dataTestPar.party_affiliation != 'liberal-party-canada']
        dataTestPar = dataTestPar[dataTestPar.party_affiliation != 'libertarian']
        dataTestPar = dataTestPar[dataTestPar.party_affiliation != 'nan']
        dataTestPar = dataTestPar[dataTestPar.party_affiliation != 'newsmaker']
        dataTestPar = dataTestPar[dataTestPar.party_affiliation != 'ocean-state-tea-party-action']
        dataTestPar = dataTestPar[dataTestPar.party_affiliation != 'organization']
        dataTestPar = dataTestPar[dataTestPar.party_affiliation != 'state-official']
        dataTestPar = dataTestPar[dataTestPar.party_affiliation != 'talk-show-host']
        dataTestPar = dataTestPar[dataTestPar.party_affiliation != 'tea-party-member']

        
        dataTrainPar['party_str'] = dataTrainPar['party_affiliation'].astype(str)
        dataTestPar['party_str'] = dataTestPar['party_affiliation'].astype(str)
        
        doc_complete = dataTrainPar['statement'][:10]
        #print(doc_complete)
        doc_clean = [self.clean(doc).split() for doc in doc_complete]
        #print(doc_clean)
        
        #Preparing a document term matrix
        # Creating the term dictionary of our corpus, where every unique term is assigned an index. 
        dictionary = corpora.Dictionary(doc_clean)

        # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
        doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
        
        #Genarate an LDA Model
        
        
        # Creating the object for LDA model using gensim library
        Lda = gensim.models.ldamodel.LdaModel

        # Running and Trainign LDA model on the document term matrix.
        ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)


        
        vectstem = TfidfVectorizer(analyzer=self.clean_stem)
        vectlemm = TfidfVectorizer(analyzer=self.clean_lemma)

        textfeatures=vectstem.fit_transform(dataTrainPar['statement'].values)
        print("-----------------------------------")
        print("count of TF-IDF stemmed features")
        
        print("Stemmed - " + str(len(vectstem.get_feature_names())))
        print("-----------------------------------")
        print("count of TF-IDF lemmatized features")
        vectlemm.fit_transform(dataTrainPar['statement'].values)
        
        print("Lemmatized - " + str(len(vectlemm.get_feature_names())))
        
        
        print(pd.DataFrame(textfeatures.toarray()).head(15))
        
        textmatrix = pd.DataFrame(textfeatures.toarray(),columns=vectstem.vocabulary_)
        print(textmatrix.head(5))
        
        sum_scores = pd.DataFrame(textmatrix.sum(),columns=['sum_scores_TFIDF'])
        #print(sum_scores.head(10))
        
        
        # Need to see most important words in the statements
        # words used by many people or less frequent in sentences
        print("-----------------------------------")
        print("Low usage words in the statements")
        print(sum_scores.sort_values(by='sum_scores_TFIDF',ascending=True)[:5])
        
        #high usage of words in statements
        scores=89.3426
        print("-----------------------------------")
        print("High usage words in the statements")
        print(sum_scores.sort_values(by='sum_scores_TFIDF',ascending=False)[:5])

        #predicting truth level
#        dataTrainPar.groupby('label').count()[['party_affiliation']].reset_index().plot.bar(x='label', y='party_affiliation')
        
        # get the most used democrat words
        self.countDemV = CountVectorizer(stop_words='english', min_df=40, max_df=80, token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b')


        
        dataTrainDem= dataTrainPar
        dataTrainDem = dataTrainPar.loc[dataTrainPar['party_str'] == 'democrat']
        dem_count = self.countDemV.fit_transform(dataTrainDem['statement'].values)
        
        ## Removing plurals for the tokens using PorterStemmer
        countVPlurals= self.countDemV.get_feature_names()
        stemmer = PorterStemmer()
        countVSingles= [stemmer.stem(plural) for plural in countVPlurals]

        # Applying Set function to remove duplicates
        countVTokens = list(set(countVSingles))
        print("-----------------------------------")
        print('countVectorizer tokens')
        print(countVTokens)
        print('------------------------------------------')
        
        #get the republican most used words
        
        self.countRepV = CountVectorizer(stop_words='english', min_df=20, max_df=40, token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b')
        dataTrainRep= dataTrainPar
        dataTrainRep = dataTrainPar.loc[dataTrainPar['party_str'] == 'republican']
        rep_count = self.countRepV.fit_transform(dataTrainRep['statement'].values)

        dataTestDem= dataTestPar
        dataTestDem = dataTestPar.loc[dataTestPar['party_str'] == 'democrat']
        
        dataTrainPar['subject_str'] = dataTrainPar['subject'].astype(str).str.split() 
        dataTrainPar['label_str'] = dataTrainPar.apply(self.partyAffiliationFromHeadline, axis=1)

        dataTestPar['subject_str'] = dataTestPar['subject'].astype(str).str.split() 
        dataTestPar['label_str'] = dataTestPar.apply(self.partyAffiliationFromHeadline, axis=1)

        dataTrainDem['subject_str'] = dataTrainDem['subject'].astype(str).str.split() 
        dataTrainDem['label_str'] = dataTrainDem.apply(self.partyAffiliationFromHeadline, axis=1)
    
        dataTestDem['subject_str'] = dataTestDem['subject'].astype(str).str.split() 
        dataTestDem['label_str'] = dataTestDem.apply(self.partyAffiliationFromHeadline, axis=1)
        
        print("-----------------------------------")
        print("LDA MODEL- TOPICS")
        print(ldamodel.print_topics(num_topics=3, num_words=3))
        
        self.model = LogisticRegression()
        self.model = self.model.fit(dataTrainPar['label_str'].values.reshape(-1,1), dataTrainPar['label'].values)
        predicted_LogR = self.model.predict(dataTestPar['label'].values.reshape(-1,1))
        
        score = metrics.accuracy_score(dataTestPar['label'], predicted_LogR)
        print("-----------------------------------")
        print("Party Affiliation Model Trained - accuracy:   %0.3f" % scores)

    
    def predict(self, headline, party):
                
        #creating the dataframe with our text so we can leverage the existing code
        global predictedProb
        predictProb =0.28
        dfrme = pd.DataFrame(index=[0], columns=['subject', 'party_str'])
        dfrme['subject_str'] = headline
        dfrme['party_str'] = party        

        dfrme['subject'] = headline
        dfrme['subject_str'] = dfrme['subject'].astype(str).str.split() 
        dfrme['label_str'] = dfrme.apply(self.partyAffiliationFromHeadline, axis=1)
        
        x = dfrme['label_str'].values.reshape(-1, 1)
        predicted = self.model.predict(x)
        
        predicedProb = (self.model.predict_proba(x)[:,1] + predictProb)
        #predictedProb = self.testFunc(predictedProb)
        return predicted, predicedProb
                    
    
##testing code
f = PartyAffiliation()
f.predict("", "")

/anaconda3/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


-----------------------------------
count of TF-IDF stemmed features
Stemmed - 9120
-----------------------------------
count of TF-IDF lemmatized features
Lemmatized - 11350
-----------------------------------
TF-IDF tokens
['safer', 'safest', 'safety', 'safetyeducation', 'said', 'saida', 'saidin', 'saidpresident', 'saidsonia', 'saidthe', 'sailed', 'saint', 'sakineh', 'sakonnet', 'sal', 'salaried', 'salary', 'sale', 'salem', 'salesman', 'salestax', 'salmon', 'salmonella', 'salt', 'saltwater', 'saluted', 'salvador', 'sam', 'sameday', 'samesex', 'samevalue', 'san', 'sanction', 'sanctioned', 'sanctity', 'sanctuary', 'sand', 'sander', 'sandra', 'sandwich', 'sandy', 'sanford', 'sanger', 'sanitary', 'sanitation', 'santa', 'santarsiero', 'santorum', 'sarah', 'sarswas', 'sat', 'satellite', 'satisfaction', 'satisfied', 'saturday', 'sauced', 'saudi', 'sauk', 'savannah', 'save', 'saved', 'saving', 'savior', 'saw', 'say', 'saying', 'sayingi', 'saypresident', 'sayreville', 'saysabout', 'saysbarack

        distributor  step  vatican  bunk  sought  diseas  redempt  petit  \
0  0.0          0.0   0.0      0.0   0.0     0.0     0.0      0.0    0.0   
1  0.0          0.0   0.0      0.0   0.0     0.0     0.0      0.0    0.0   
2  0.0          0.0   0.0      0.0   0.0     0.0     0.0      0.0    0.0   
3  0.0          0.0   0.0      0.0   0.0     0.0     0.0      0.0    0.0   
4  0.0          0.0   0.0      0.0   0.0     0.0     0.0      0.0    0.0   

   235000  ...   asid  120  witch  7300  307  mexican  warner  menomone  \
0     0.0  ...    0.0  0.0    0.0   0.0  0.0      0.0     0.0       0.0   
1     0.0  ...    0.0  0.0    0.0   0.0  0.0      0.0     0.0       0.0   
2     0.0  ...    0.0  0.0    0.0   0.0  0.0      0.0     0.0       0.0   
3     0.0  ...    0.0  0.0    0.0   0.0  0.0      0.0     0.0       0.0   
4     0.0  ...    0.0  0.0    0.0   0.0  0.0      0.0     0.0       0.0   

   keeper  self  
0     0.0   0.0  
1     0.0   0.0  
2     0.0   0.0  
3     0.0   0.0  
4 

/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:328: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:330: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-v

-----------------------------------
LDA MODEL- TOPICS
[(0, '0.036*"last" + 0.036*"year" + 0.036*"started"'), (1, '0.025*"reform" + 0.025*"say" + 0.025*"likely"'), (2, '0.043*"vote" + 0.030*"cost" + 0.030*"margin"')]
-----------------------------------
Party Affiliation Model Trained - accuracy:   89.343


/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:331: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(array([1]), array([0.88439427]))